In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import emoji
from nltk.tokenize import word_tokenize
from collections import Counter, OrderedDict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [2]:
path = "../Dataset FFF/fff.csv"

In [3]:
df=pd.read_csv(path)
df.head()

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,replies_count,retweets_count,likes_count,link,retweet,reply_to
0,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",2,54,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof..."
1,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[]
2,fridaysforfuture,['fridaysforfuture'],2022-07-10,126978307,kl_stone,Klaus Steinfelder,SERVICETWEET: Zur aktuellen Hetze der BLÖD-Ze...,de,[],2,5,8,https://twitter.com/Kl_Stone/status/1546221657...,False,[]
3,fridayforfuture,['fridayforfuture'],2022-07-10,403735368,hahnflo,Florian Hahn,"Wer erklärt #FridayForFuture, dass es ohne NAT...",de,[],127,65,679,https://twitter.com/hahnflo/status/15461339384...,False,[]
4,"klimakrise, govegan, fridaysforfuture","['klimakrise', 'govegan', 'fridaysforfuture']",2022-07-10,1529883123881857031,vegancardio,Der vegane Kardiologe,"Was sind das eigentlich für Menschen, die sich...",de,[],79,25,348,https://twitter.com/VeganCardio/status/1546127...,False,[]


In [4]:
len(df["tweet"])

23504

### Data cleaning

In [5]:
punctaction = re.sub("@", "", string.punctuation)
punctaction

'!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'

In [6]:
def clean_text(corpus):
    clear_text = []
    for phrase in corpus:
        # Replace the mentions with the tag @mention
        phrase = re.sub("@\S+", "@mention ", phrase)
        # Replace URL with the tag @url
        phrase = re.sub("https*\S+", " @url", phrase)
        # Replace hashtag with the tag @hashtag
        phrase = re.sub("#\S+", " @hashtag", phrase)
        # Remove strange characters
        phrase = re.sub("\x89Ûª|\x89Û\x9d|\x89Ûª|\x89Û÷|\x89Û_", "", phrase)
    
        # Replace strange emoji with the tag @emoji
        phrase = re.sub("\x89\S+", " @emoji", phrase)
        phrase = re.sub("\x95\S+", " @emoji", phrase)
        phrase = re.sub("_Ù\S+", " @emoji", phrase)
        phrase = re.sub("÷\S+", " @emoji", phrase)
        # Replace light emoji like :-D ;D ;p
        phrase = re.sub(":+[';()-/|DPXp]\S*|;+['()-/|DPXp]\S*", " @emoji", phrase)

        # Punctuaction removal
        phrase = re.sub('[%s]' % re.escape(punctaction), ' ', phrase)
        # Replace numbers with the tag @number
        phrase = re.sub("\d+", " @number ", phrase)
        # Convert to lowercase
        phrase = phrase.lower()
        # Replace the over spaces
        phrase = re.sub('\s{2,}', " ", phrase)
        clear_text.append([phrase])
    return clear_text

In [7]:
cleared_text = clean_text(df["tweet"])

In [8]:
cleared_text

[['@mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @mention @hashtag are doomed integrated @hashtag cheapest @hashtag @hashtag costs falling fast kindly share @url @hashtag @hashtag @hashtag @hashtag @hashtag @hashtag @hashtag @hashtag @mention @mention @mention @mention @url'],
 ['eltern wissen wer hier dumm ist @hashtag ist es nicht sorry für radikalen spinner in timeline🤷🏾 @url'],
 ['servicetweet zur aktuellen hetze der blöd zeitung wenn sich @hashtag sverige zur legitimen innerschwedischen diskussion über die beendigung jahrhundertealter neutralität äußert dann ist das weder linksextrem noch stellt es die solidarität mit der ukrain

In [9]:
df['cleared_text'] = np.array(cleared_text)

In [10]:
df['cleared_text']

0        @mention @mention @mention @mention @mention @...
1        eltern wissen wer hier dumm ist @hashtag ist e...
2        servicetweet zur aktuellen hetze der blöd zeit...
3        wer erklärt @hashtag dass es ohne nato keine s...
4        was sind das eigentlich für menschen die sich ...
                               ...                        
23499    foto von heute aus berlin @hashtag vielen dank...
23500    the world is listening and the message is clea...
23501    beeindruckend wenn andere auf wissenschaftler ...
23502    @mention @mention @mention @mention @mention @...
23503    verkehrschaos in der city auf dem ring ist der...
Name: cleared_text, Length: 23504, dtype: object

## Tokenization

In [11]:


"""
word_tokens = []
for phrase in df["cleared_text"]:
    word_tokens.append(word_tokenize(phrase))
"""



'\nword_tokens = []\nfor phrase in df["cleared_text"]:\n    word_tokens.append(word_tokenize(phrase))\n'

In [12]:
word_tokens = []
for phrase in df["cleared_text"]:
    word_tokens.append(phrase.split())

In [13]:
word_tokens

[['@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@hashtag',
  'are',
  'doomed',
  'integrated',
  '@hashtag',
  'cheapest',
  '@hashtag',
  '@hashtag',
  'costs',
  'falling',
  'fast',
  'kindly',
  'share',
  '@url',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@url'],
 ['

### Remove Stopwords:

In [14]:
stop_words = stopwords.words('english')+ stopwords.words('italian')+ stopwords.words('german')+ stopwords.words('swedish')

In [15]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [16]:
# Removing quotes
stop_words_wq = []
for w in stop_words:
    w = re.sub("\'", "", w)
    stop_words_wq.append(w)

In [17]:
stop_words_wq

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'youre',
 'youve',
 'youll',
 'youd',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'shes',
 'her',
 'hers',
 'herself',
 'it',
 'its',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'thatll',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'fe

In [18]:
no_sw=[]
for phrase in word_tokens:
    no_sw_tmp=[]
    for word in phrase:
        if word not in stop_words_wq:
            no_sw_tmp.append(word)
    no_sw.append(no_sw_tmp)

In [19]:
no_sw

[['@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@hashtag',
  'doomed',
  'integrated',
  '@hashtag',
  'cheapest',
  '@hashtag',
  '@hashtag',
  'costs',
  'falling',
  'fast',
  'kindly',
  'share',
  '@url',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@url'],
 ['eltern',


In [20]:
df["no_sw"] = np.array(no_sw)

<ipython-input-20-43fd746dccd2>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df["no_sw"] = np.array(no_sw)


In [21]:
df.head()

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,replies_count,retweets_count,likes_count,link,retweet,reply_to,cleared_text,no_sw
0,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",2,54,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof...",@mention @mention @mention @mention @mention @...,"[@mention, @mention, @mention, @mention, @ment..."
1,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[],eltern wissen wer hier dumm ist @hashtag ist e...,"[eltern, wissen, wer, dumm, @hashtag, sorry, r..."
2,fridaysforfuture,['fridaysforfuture'],2022-07-10,126978307,kl_stone,Klaus Steinfelder,SERVICETWEET: Zur aktuellen Hetze der BLÖD-Ze...,de,[],2,5,8,https://twitter.com/Kl_Stone/status/1546221657...,False,[],servicetweet zur aktuellen hetze der blöd zeit...,"[servicetweet, aktuellen, hetze, blöd, zeitung..."
3,fridayforfuture,['fridayforfuture'],2022-07-10,403735368,hahnflo,Florian Hahn,"Wer erklärt #FridayForFuture, dass es ohne NAT...",de,[],127,65,679,https://twitter.com/hahnflo/status/15461339384...,False,[],wer erklärt @hashtag dass es ohne nato keine s...,"[wer, erklärt, @hashtag, nato, sicherheit, sic..."
4,"klimakrise, govegan, fridaysforfuture","['klimakrise', 'govegan', 'fridaysforfuture']",2022-07-10,1529883123881857031,vegancardio,Der vegane Kardiologe,"Was sind das eigentlich für Menschen, die sich...",de,[],79,25,348,https://twitter.com/VeganCardio/status/1546127...,False,[],was sind das eigentlich für menschen die sich ...,"[eigentlich, menschen, heutigen, zeit, guten, ..."


## Remove short words

In [22]:
#Matteo mette parole non più corte di un carattere, noi ne mettiamo 2
no_sw_ll = []
for i in df["no_sw"]:
    tmp = []
    for word in i:
        if len(word) > 2:
            tmp.append(word)
    no_sw_ll.append(tmp)

In [23]:
no_sw_ll

[['@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@hashtag',
  'doomed',
  'integrated',
  '@hashtag',
  'cheapest',
  '@hashtag',
  '@hashtag',
  'costs',
  'falling',
  'fast',
  'kindly',
  'share',
  '@url',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@hashtag',
  '@mention',
  '@mention',
  '@mention',
  '@mention',
  '@url'],
 ['eltern',


In [24]:
df["not_shortw"] = no_sw_ll

In [25]:
df.head()

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,replies_count,retweets_count,likes_count,link,retweet,reply_to,cleared_text,no_sw,not_shortw
0,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",2,54,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof...",@mention @mention @mention @mention @mention @...,"[@mention, @mention, @mention, @mention, @ment...","[@mention, @mention, @mention, @mention, @ment..."
1,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[],eltern wissen wer hier dumm ist @hashtag ist e...,"[eltern, wissen, wer, dumm, @hashtag, sorry, r...","[eltern, wissen, wer, dumm, @hashtag, sorry, r..."
2,fridaysforfuture,['fridaysforfuture'],2022-07-10,126978307,kl_stone,Klaus Steinfelder,SERVICETWEET: Zur aktuellen Hetze der BLÖD-Ze...,de,[],2,5,8,https://twitter.com/Kl_Stone/status/1546221657...,False,[],servicetweet zur aktuellen hetze der blöd zeit...,"[servicetweet, aktuellen, hetze, blöd, zeitung...","[servicetweet, aktuellen, hetze, blöd, zeitung..."
3,fridayforfuture,['fridayforfuture'],2022-07-10,403735368,hahnflo,Florian Hahn,"Wer erklärt #FridayForFuture, dass es ohne NAT...",de,[],127,65,679,https://twitter.com/hahnflo/status/15461339384...,False,[],wer erklärt @hashtag dass es ohne nato keine s...,"[wer, erklärt, @hashtag, nato, sicherheit, sic...","[wer, erklärt, @hashtag, nato, sicherheit, sic..."
4,"klimakrise, govegan, fridaysforfuture","['klimakrise', 'govegan', 'fridaysforfuture']",2022-07-10,1529883123881857031,vegancardio,Der vegane Kardiologe,"Was sind das eigentlich für Menschen, die sich...",de,[],79,25,348,https://twitter.com/VeganCardio/status/1546127...,False,[],was sind das eigentlich für menschen die sich ...,"[eigentlich, menschen, heutigen, zeit, guten, ...","[eigentlich, menschen, heutigen, zeit, guten, ..."


## Lemmatization:

In [26]:
lemmatizer = WordNetLemmatizer()

In [27]:
def lemmatization(data, lemmatizer):
    new_words_lemm=[]
    for row in data:
        new_words_lemm_tmp=[]
        for word in row:
            lemm_word = lemmatizer.lemmatize(word)
            lemm_word = lemmatizer.lemmatize(lemm_word, pos='v')
            new_words_lemm_tmp.append(lemm_word)
        new_words_lemm.append(new_words_lemm_tmp)
    return new_words_lemm

In [28]:
df["lemmatized"] = lemmatization(df["not_shortw"], lemmatizer)


In [29]:
df.head()

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,replies_count,retweets_count,likes_count,link,retweet,reply_to,cleared_text,no_sw,not_shortw,lemmatized
0,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",2,54,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof...",@mention @mention @mention @mention @mention @...,"[@mention, @mention, @mention, @mention, @ment...","[@mention, @mention, @mention, @mention, @ment...","[@mention, @mention, @mention, @mention, @ment..."
1,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],21,54,595,https://twitter.com/parents4future/status/1546...,False,[],eltern wissen wer hier dumm ist @hashtag ist e...,"[eltern, wissen, wer, dumm, @hashtag, sorry, r...","[eltern, wissen, wer, dumm, @hashtag, sorry, r...","[eltern, wissen, wer, dumm, @hashtag, sorry, r..."
2,fridaysforfuture,['fridaysforfuture'],2022-07-10,126978307,kl_stone,Klaus Steinfelder,SERVICETWEET: Zur aktuellen Hetze der BLÖD-Ze...,de,[],2,5,8,https://twitter.com/Kl_Stone/status/1546221657...,False,[],servicetweet zur aktuellen hetze der blöd zeit...,"[servicetweet, aktuellen, hetze, blöd, zeitung...","[servicetweet, aktuellen, hetze, blöd, zeitung...","[servicetweet, aktuellen, hetze, blöd, zeitung..."
3,fridayforfuture,['fridayforfuture'],2022-07-10,403735368,hahnflo,Florian Hahn,"Wer erklärt #FridayForFuture, dass es ohne NAT...",de,[],127,65,679,https://twitter.com/hahnflo/status/15461339384...,False,[],wer erklärt @hashtag dass es ohne nato keine s...,"[wer, erklärt, @hashtag, nato, sicherheit, sic...","[wer, erklärt, @hashtag, nato, sicherheit, sic...","[wer, erklärt, @hashtag, nato, sicherheit, sic..."
4,"klimakrise, govegan, fridaysforfuture","['klimakrise', 'govegan', 'fridaysforfuture']",2022-07-10,1529883123881857031,vegancardio,Der vegane Kardiologe,"Was sind das eigentlich für Menschen, die sich...",de,[],79,25,348,https://twitter.com/VeganCardio/status/1546127...,False,[],was sind das eigentlich für menschen die sich ...,"[eigentlich, menschen, heutigen, zeit, guten, ...","[eigentlich, menschen, heutigen, zeit, guten, ...","[eigentlich, menschen, heutigen, zeit, guten, ..."


In [30]:
new_col = []
for words in df["lemmatized"]:
    string_col = ""
    for word in words:
        string_col = string_col + word + " "
    new_col.append(string_col)

In [31]:
df["preprocessed_text"] = new_col

In [32]:
df['length'] = [len(t) for t in df.preprocessed_text]
df.head()

,clean_tags,hashtags,date,user_id,username,name,tweet,language,mentions,replies_count,...,likes_count,link,retweet,reply_to,cleared_text,no_sw,not_shortw,lemmatized,preprocessed_text,length
0,"fossilfuels, wind, solar, green, hydrogen, ene...","['fossilfuels', 'wind', 'solar', 'green', 'hyd...",2022-07-10,59386332,arikring,Arik Ring - Energy Engineering Expert,@ProfStrachan @ECOWARRIORSS @ChristineMilne @m...,en,"[{'screen_name': 'guytaur', 'name': 'scott. tr...",2,...,59,https://twitter.com/arikring/status/1546251358...,False,"[{'screen_name': 'ProfStrachan', 'name': 'Prof...",@mention @mention @mention @mention @mention @...,"[@mention, @mention, @mention, @mention, @ment...","[@mention, @mention, @mention, @mention, @ment...","[@mention, @mention, @mention, @mention, @ment...",@mention @mention @mention @mention @mention @...,620
1,fridaysforfuture,['fridaysforfuture'],2022-07-10,1096766566279073792,parents4future,Parents For Future #EndFossilFuels,"Eltern wissen, wer hier dumm ist. #FridaysForF...",de,[],21,...,595,https://twitter.com/parents4future/status/1546...,False,[],eltern wissen wer hier dumm ist @hashtag ist e...,"[eltern, wissen, wer, dumm, @hashtag, sorry, r...","[eltern, wissen, wer, dumm, @hashtag, sorry, r...","[eltern, wissen, wer, dumm, @hashtag, sorry, r...",eltern wissen wer dumm @hashtag sorry radikale...,72
2,fridaysforfuture,['fridaysforfuture'],2022-07-10,126978307,kl_stone,Klaus Steinfelder,SERVICETWEET: Zur aktuellen Hetze der BLÖD-Ze...,de,[],2,...,8,https://twitter.com/Kl_Stone/status/1546221657...,False,[],servicetweet zur aktuellen hetze der blöd zeit...,"[servicetweet, aktuellen, hetze, blöd, zeitung...","[servicetweet, aktuellen, hetze, blöd, zeitung...","[servicetweet, aktuellen, hetze, blöd, zeitung...",servicetweet aktuellen hetze blöd zeitung @has...,202
3,fridayforfuture,['fridayforfuture'],2022-07-10,403735368,hahnflo,Florian Hahn,"Wer erklärt #FridayForFuture, dass es ohne NAT...",de,[],127,...,679,https://twitter.com/hahnflo/status/15461339384...,False,[],wer erklärt @hashtag dass es ohne nato keine s...,"[wer, erklärt, @hashtag, nato, sicherheit, sic...","[wer, erklärt, @hashtag, nato, sicherheit, sic...","[wer, erklärt, @hashtag, nato, sicherheit, sic...",wer erklärt @hashtag nato sicherheit sicherhei...,65
4,"klimakrise, govegan, fridaysforfuture","['klimakrise', 'govegan', 'fridaysforfuture']",2022-07-10,1529883123881857031,vegancardio,Der vegane Kardiologe,"Was sind das eigentlich für Menschen, die sich...",de,[],79,...,348,https://twitter.com/VeganCardio/status/1546127...,False,[],was sind das eigentlich für menschen die sich ...,"[eigentlich, menschen, heutigen, zeit, guten, ...","[eigentlich, menschen, heutigen, zeit, guten, ...","[eigentlich, menschen, heutigen, zeit, guten, ...",eigentlich menschen heutigen zeit guten gewiss...,177


### Store preprocessed data

In [33]:
def delet_tag(corpus):
    clear_text = []
    for phrase in corpus:
        # Replace the @
        phrase = re.sub("@\S+", "", phrase)
        phrase = re.sub("#\S+", "", phrase)
        #phrase = re.sub(emoji.emojize,"",phrase)
        clear_text.append(phrase)
    return clear_text

In [34]:
df["meaning"] = delet_tag(df["preprocessed_text"])

In [35]:
df= df[['clean_tags', 'meaning']]

In [36]:
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [37]:
df.head(10)

,clean_tags,meaning
0,"fossilfuels, wind, solar, green, hydrogen, ene...",...
1,fridaysforfuture,eltern wissen wer dumm sorry radikalen spinne...
2,fridaysforfuture,servicetweet aktuellen hetze bld zeitung sver...
3,fridayforfuture,wer erklrt nato sicherheit sicherheit future ...
4,"klimakrise, govegan, fridaysforfuture",eigentlich menschen heutigen zeit guten gewiss...
5,hitzewelle,brigens aktionen aufregt wer klaren verstand...
6,fridaysforfuture,zeigt mal drauf habt rettet klima
7,"fridaysforfuture, peoplenotprofit",climate strike school yesterday photo take see...
8,"climatecrisis, climatestrike, climateaction, c...",climate justice approach promote transformativ...
9,"peoplepower, actnow, climateaction, climatecri...",older generation fail young pay full future ge...
